In [ ]:
import pandas as pd # Import pandas librairy: Pandas is a kind of super Excel in Python
import matplotlib.pyplot as plt # matplotlib is a general purpose librairy for graphics
from scipy import stats # scipy stats is a librairy of statistical funstions
import numpy as np # numpy is a librairy to manipulate matrix / tables
from pandas import Grouper # to group data and make cool graphs
import seaborn as sns # to plot data analysis graphs
from sklearn.model_selection import train_test_split, learning_curve # for machine learning
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler # for machine learning
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import tensorflow as tf
import tensorflow.keras.regularizers

In [ ]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
path = "/content/drive/My Drive"
os.chdir(path)

In [ ]:
dir = 'Projet informatique/data/'

In [ ]:
data = pd.read_pickle(dir + 'cleaned_data.pkl')

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7093443 entries, 0 to 2459541
Data columns (total 21 columns):
 #   Column                                    Dtype          
---  ------                                    -----          
 0   Transaction date                          datetime64[ns] 
 1   Transaction type                          object         
 2   Transaction cost                          float64        
 3   Adress number                             float64        
 4   Adress number extension                   object         
 5   Adress street name                        object         
 6   Postcode                                  float64        
 7   Town                                      object         
 8   Department                                object         
 9   Type of property                          object         
 10  Built surface                             float64        
 11  Number of rooms                           float64        
 12  

In [ ]:
data.head()

,Transaction date,Transaction type,Transaction cost,Adress number,Adress number extension,Adress street name,Postcode,Town,Department,Type of property,Built surface,Number of rooms,Type of ground,Ground surface,Longitude,Latitude,Cost per m2,Delta days,Dépendance,Local industriel. commercial ou assimilé,Maison
0,2014-01-09,Vente,251500.0,532.0,NaN,RTE DE MEZERIAT,1310.0,Polliat,01,Maison,147.0,5.0,sols,1501.0,5.119191,46.247576,1711.0,8 days,0,0,1
2,2014-01-09,Vente,174500.0,22.0,NaN,BD MAL LECLERC,1000.0,Bourg-en-Bresse,01,Appartement,80.0,2.0,NaN,0.0,5.218208,46.204017,2181.0,8 days,0,0,0
4,2014-01-02,Vente,157500.0,1369.0,NaN,RTE DE STRASBOURG,1440.0,Viriat,01,Maison,103.0,4.0,sols,1569.0,5.262756,46.236276,1529.0,1 days,0,0,1
7,2014-01-07,Vente,90000.0,4.0,NaN,RUE GEN DELESTRAINT,1000.0,Bourg-en-Bresse,01,Appartement,61.0,2.0,NaN,0.0,5.219821,46.200397,1475.0,6 days,0,0,0
11,2014-01-14,Vente,95000.0,3.0,NaN,RUE DES TULIPES,1000.0,Bourg-en-Bresse,01,Appartement,58.0,3.0,NaN,0.0,5.210812,46.204702,1638.0,13 days,0,0,0


In [ ]:
data.isna().sum()

Transaction date                                  0
Transaction type                                  0
Transaction cost                                  0
Adress number                                 55711
Adress number extension                     6617091
Adress street name                             3498
Postcode                                       3847
Town                                              0
Department                                        0
Type of property                                  0
Built surface                                     0
Number of rooms                                   0
Type of ground                              2829799
Ground surface                                    0
Longitude                                    159220
Latitude                                     159220
Cost per m2                                       0
Delta days                                        0
Dépendance                                        0
Local indust

In [ ]:
data = data.dropna('index', subset = ['Postcode', 'Adress number', 'Longitude', 'Latitude'])

In [ ]:
deep_learning_labels = ['Transaction date', 'Transaction cost', 'Adress number', 'Postcode', 'Type of property', 'Built surface', 'Number of rooms', 'Ground surface', 'Longitude', 'Latitude']

data = data[deep_learning_labels]

print(data.columns)

Index(['Transaction date', 'Transaction cost', 'Adress number', 'Postcode',
       'Type of property', 'Built surface', 'Number of rooms',
       'Ground surface', 'Longitude', 'Latitude'],
      dtype='object')


In [ ]:
data.isna().sum()

Transaction date    0
Transaction cost    0
Adress number       0
Postcode            0
Type of property    0
Built surface       0
Number of rooms     0
Ground surface      0
Longitude           0
Latitude            0
dtype: int64

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6876357 entries, 0 to 2459541
Data columns (total 10 columns):
 #   Column            Dtype         
---  ------            -----         
 0   Transaction date  datetime64[ns]
 1   Transaction cost  float64       
 2   Adress number     float64       
 3   Postcode          float64       
 4   Type of property  object        
 5   Built surface     float64       
 6   Number of rooms   float64       
 7   Ground surface    float64       
 8   Longitude         float64       
 9   Latitude          float64       
dtypes: datetime64[ns](1), float64(8), object(1)
memory usage: 577.1+ MB


In [ ]:
# change Transaction date dtype to num
reference_time = pd.Timestamp("01.01.2014 00:00:00").value
data['Transaction date'] = pd.to_datetime(data['Transaction date']).astype(np.int64)
data['Transaction date'] = (data['Transaction date'] - reference_time) / 86400000000000

data.head()

,Transaction date,Transaction cost,Adress number,Postcode,Type of property,Built surface,Number of rooms,Ground surface,Longitude,Latitude
0,8.0,251500.0,532.0,1310.0,Maison,147.0,5.0,1501.0,5.119191,46.247576
2,8.0,174500.0,22.0,1000.0,Appartement,80.0,2.0,0.0,5.218208,46.204017
4,1.0,157500.0,1369.0,1440.0,Maison,103.0,4.0,1569.0,5.262756,46.236276
7,6.0,90000.0,4.0,1000.0,Appartement,61.0,2.0,0.0,5.219821,46.200397
11,13.0,95000.0,3.0,1000.0,Appartement,58.0,3.0,0.0,5.210812,46.204702


In [ ]:
# change Type of property to num (Appartement = 0, Maison = 1)
data['Type of property']  = (data['Type of property'] == "Maison").astype(float)

In [ ]:
data.head()

,Transaction date,Transaction cost,Adress number,Postcode,Type of property,Built surface,Number of rooms,Ground surface,Longitude,Latitude
0,8.0,251500.0,532.0,1310.0,1.0,147.0,5.0,1501.0,5.119191,46.247576
2,8.0,174500.0,22.0,1000.0,0.0,80.0,2.0,0.0,5.218208,46.204017
4,1.0,157500.0,1369.0,1440.0,1.0,103.0,4.0,1569.0,5.262756,46.236276
7,6.0,90000.0,4.0,1000.0,0.0,61.0,2.0,0.0,5.219821,46.200397
11,13.0,95000.0,3.0,1000.0,0.0,58.0,3.0,0.0,5.210812,46.204702


In [ ]:
data.to_pickle(dir + 'DL_data.pkl')

# **Deep Learning**




In [ ]:
data = pd.read_pickle(dir + 'DL_data.pkl')

In [ ]:
data = data.loc[((data['Number of rooms']>0) &
                 (data['Transaction cost']<=2e6) &
                 (data['Transaction cost']>=2e4)), :]

In [ ]:
# create final dataset for the Deep Learning
x_index = ['Transaction date', 'Built surface', 'Ground surface', 'Longitude', 'Latitude']

y = np.array(data['Transaction cost'])
X = np.array(data[x_index])

print('X_shape')
print(X.shape)
print('y_shape')
print(y.shape)

X_shape
(6361288, 5)
y_shape
(6361288,)


In [ ]:
# standardization and split to train / test set

In [ ]:
X = StandardScaler().fit_transform(X)
y = y
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [ ]:
print(X_train.shape)

(4452901, 5)


# my_model_0412

In [ ]:
drop_prob = 0.4
weight_penalty_1 = 1e-5
weight_penalty_2 = 1e-5

inputs = tf.keras.layers.Input(shape=(X.shape[1]))
layer1 = tf.keras.layers.Dense(1024, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(weight_penalty_2),
                    activity_regularizer=tf.keras.regularizers.l2(weight_penalty_2))(inputs)

layer2 = tf.keras.layers.BatchNormalization()(layer1)
layer3 = tf.keras.layers.Dropout(drop_prob)(layer2)                    
layer4 = tf.keras.layers.Dense(1024, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(weight_penalty_2),
                    activity_regularizer=tf.keras.regularizers.l2(weight_penalty_2))(layer3)

layer5 = tf.keras.layers.BatchNormalization()(layer4)
layer6 = tf.keras.layers.Dropout(drop_prob)(layer5)
layer7 = tf.keras.layers.Dense(1024, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(weight_penalty_2),
                    activity_regularizer=tf.keras.regularizers.l2(weight_penalty_2))(layer6)

output = tf.keras.layers.Dense(1)(layer7)
model = tf.keras.Model(inputs=inputs, outputs=output)

In [ ]:
boundaries = [50, 200, 500, 1000]
values = [0.1, 0.05, 0.01, 0.005, 0.001]
learning_rate_fn = tf.keras.optimizers.schedules.PiecewiseConstantDecay(boundaries, values)
model_optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate_fn)
model.compile(
    loss='mape',
    optimizer=model_optimizer,
    metrics=['mae']
)
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 6)]               0         
_________________________________________________________________
dense_4 (Dense)              (None, 1024)              7168      
_________________________________________________________________
batch_normalization_2 (Batch (None, 1024)              4096      
_________________________________________________________________
dropout_2 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
batch_normalization_3 (Batch (None, 1024)              4096      
_________________________________________________________________
dropout_3 (Dropout)          (None, 1024)              0   

In [ ]:
from datetime import datetime
logdir = "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)
EarlyStoppingCallback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)

In [ ]:
training_history = model.fit(
    X_train, # input
    y_train, # output
    batch_size = 8192,
    epochs = 2000,
    validation_split=0.2,
    callbacks=[EarlyStoppingCallback, tensorboard_callback],
)

Epoch 1/2000
179/179 [==============================] - 11s 53ms/step - loss: 67.1516 - mae: 154612.0278 - val_loss: 45.0788 - val_mae: 109091.7031
Epoch 2/2000
179/179 [==============================] - 9s 49ms/step - loss: 43.4686 - mae: 107063.2304 - val_loss: 42.6448 - val_mae: 106324.4141
Epoch 3/2000
179/179 [==============================] - 9s 50ms/step - loss: 41.3826 - mae: 101974.1490 - val_loss: 40.5055 - val_mae: 102363.1797
Epoch 4/2000
179/179 [==============================] - 9s 50ms/step - loss: 40.1758 - mae: 98655.3200 - val_loss: 39.1909 - val_mae: 98591.0703
Epoch 5/2000
179/179 [==============================] - 9s 50ms/step - loss: 39.5922 - mae: 97439.2181 - val_loss: 38.4734 - val_mae: 95639.8359
Epoch 6/2000
179/179 [==============================] - 9s 50ms/step - loss: 39.0456 - mae: 96303.2267 - val_loss: 37.9282 - val_mae: 95684.6953
Epoch 7/2000
179/179 [==============================] - 9s 50ms/step - loss: 38.6011 - mae: 95082.4355 - val_loss: 37.6421 

KeyboardInterrupt: ignored

In [ ]:
model.save('Projet informatique/model/my_model_0412')

INFO:tensorflow:Assets written to: Projet informatique/model/my_model_0412/assets


In [ ]:
model = tf.keras.models.load_model('Projet informatique/model/my_model_0412')

In [ ]:
prediction = model.predict(X_test)

In [ ]:
model.evaluate(X_test, y_test, batch_size=8192)

224/224 [==============================] - 6s 25ms/step - loss: 29.8884 - mae: 75925.3203


[29.888368606567383, 75925.3203125]

In [ ]:
err = np.zeros(y_test.shape[0])
relative_err = np.zeros(y_test.shape[0])
for i in range(y_test.shape[0]):
  err[i] = y_test[i] - prediction[i]
  if y_test[i] != 0:
    relative_err[i] = np.abs(err[i]) / y_test[i]
  else:
    relative_err[i] = 0

In [ ]:
relative_err.shape

(1828405,)

In [ ]:
relative_err = np.sort(relative_err)

In [ ]:
n = relative_err.shape[0]
prop = 0.8

print(int(n*prop))
print(relative_err[int(n*prop)])

1462724
0.43988372217465754


# model 0415

In [ ]:
X = StandardScaler().fit_transform(X)
y = y / 100000
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [ ]:
print(X_train.shape)

(4452901, 5)


In [ ]:
drop_prob = 0.4
weight_penalty_1 = 3e-5
weight_penalty_2 = 3e-4

inputs = tf.keras.layers.Input(shape=(X.shape[1]))
layer1 = tf.keras.layers.Dense(1024, activation='relu', activity_regularizer=tf.keras.regularizers.l2(weight_penalty_2))(inputs)

layer2 = tf.keras.layers.BatchNormalization()(layer1)
# layer3 = tf.keras.layers.Dropout(drop_prob)(layer2)                    
layer4 = tf.keras.layers.Dense(1024, activation='relu', activity_regularizer=tf.keras.regularizers.l2(weight_penalty_2))(layer2)

layer5 = tf.keras.layers.BatchNormalization()(layer4)
layer6 = tf.keras.layers.Dropout(drop_prob)(layer5)
layer7 = tf.keras.layers.Dense(1024, activation='relu', activity_regularizer=tf.keras.regularizers.l2(weight_penalty_2))(layer6)

output = tf.keras.layers.Dense(1)(layer7)
model = tf.keras.Model(inputs=inputs, outputs=output)

In [ ]:
boundaries = [50, 200, 500, 1000]
values = [0.1, 0.05, 0.01, 0.005, 0.001]
learning_rate_fn = tf.keras.optimizers.schedules.PiecewiseConstantDecay(boundaries, values)
model_optimizer = tf.keras.optimizers.Adam(learning_rate=0.03)
model.compile(
    loss='mape',
    optimizer=model_optimizer,
    metrics=['mae']
)
model.summary()

Model: "model_25"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_26 (InputLayer)        [(None, 5)]               0         
_________________________________________________________________
dense_104 (Dense)            (None, 1024)              6144      
_________________________________________________________________
batch_normalization_54 (Batc (None, 1024)              4096      
_________________________________________________________________
dense_105 (Dense)            (None, 1024)              1049600   
_________________________________________________________________
batch_normalization_55 (Batc (None, 1024)              4096      
_________________________________________________________________
dropout_54 (Dropout)         (None, 1024)              0         
_________________________________________________________________
dense_106 (Dense)            (None, 1024)              104

In [ ]:
from datetime import datetime
logdir = "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)
EarlyStoppingCallback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)

In [ ]:
training_history = model.fit(
    X_train, # input
    y_train, # output
    batch_size = 16384,
    epochs = 1000,
    validation_split=0.2,
    callbacks=[EarlyStoppingCallback, tensorboard_callback],
)
model.save('Projet informatique/model/my_model_0415')

Epoch 1/1000
218/218 [==============================] - 22s 96ms/step - loss: 822.2446 - mae: 11.0048 - val_loss: 53.5691 - val_mae: 1.1531
Epoch 2/1000
218/218 [==============================] - 20s 93ms/step - loss: 41.3486 - mae: 1.0076 - val_loss: 47.5416 - val_mae: 1.2313
Epoch 3/1000
218/218 [==============================] - 20s 91ms/step - loss: 39.8410 - mae: 0.9627 - val_loss: 41.6800 - val_mae: 0.9789
Epoch 4/1000
218/218 [==============================] - 20s 91ms/step - loss: 39.3523 - mae: 0.9454 - val_loss: 38.6361 - val_mae: 0.9315
Epoch 5/1000
218/218 [==============================] - 20s 91ms/step - loss: 38.9498 - mae: 0.9336 - val_loss: 39.5164 - val_mae: 0.8977
Epoch 6/1000
218/218 [==============================] - 20s 92ms/step - loss: 38.6585 - mae: 0.9229 - val_loss: 37.7366 - val_mae: 0.9023
Epoch 7/1000
218/218 [==============================] - 20s 91ms/step - loss: 38.2849 - mae: 0.9103 - val_loss: 37.8219 - val_mae: 0.8771
Epoch 8/1000
218/218 [==========

# Visualization

In [ ]:
nb = 1000  # max: 10000 - a smaller value speeds up visualization
X_vi = X[:nb]

X_embedded = TSNE(n_components=2).fit_transform(x_test_encoded)
# X_embedded = Isomap(10, n_components=2).fit_transform(x_test_encoded)
# X_embedded = MDS(n_components=2).fit_transform(x_test_encoded)
X_embedded.shape

plt.figure(figsize=(10, 10))
plt.set_cmap('Spectral')
plt.scatter(X_embedded[:, 0], X_embedded[:, 1], c=y_test[0:nb])
plt.colorbar()
plt.show()

# ML (pre-training with 'Longitude' and 'Latitude')

In [ ]:
data = pd.read_pickle(dir + 'DL_data.pkl')

In [ ]:
data.head()

,Transaction date,Transaction cost,Adress number,Postcode,Type of property,Built surface,Number of rooms,Ground surface,Longitude,Latitude
0,8.0,251500.0,532.0,1310.0,1.0,147.0,5.0,1501.0,5.119191,46.247576
2,8.0,174500.0,22.0,1000.0,0.0,80.0,2.0,0.0,5.218208,46.204017
4,1.0,157500.0,1369.0,1440.0,1.0,103.0,4.0,1569.0,5.262756,46.236276
7,6.0,90000.0,4.0,1000.0,0.0,61.0,2.0,0.0,5.219821,46.200397
11,13.0,95000.0,3.0,1000.0,0.0,58.0,3.0,0.0,5.210812,46.204702


In [ ]:
x_index = ['Longitude', 'Latitude']

y = np.array(data['Transaction cost'])
X = np.array(data[x_index])

print('X_shape')
print(X.shape)
print('y_shape')
print(y.shape)

X_shape
(6876357, 2)
y_shape
(6876357,)


In [ ]:
X = StandardScaler().fit_transform(X)
y = y
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [ ]:
drop_prob = 0.4
weight_penalty_1 = 1e-6
weight_penalty_2 = 1e-6

inputs = tf.keras.layers.Input(shape=(X.shape[1]))
layer1 = tf.keras.layers.Dense(16, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(weight_penalty_1),
                    activity_regularizer=tf.keras.regularizers.l2(weight_penalty_2))(inputs)

layer2 = tf.keras.layers.BatchNormalization()(layer1)
layer3 = tf.keras.layers.Dropout(drop_prob)(layer2)                    
layer4 = tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(weight_penalty_1),
                    activity_regularizer=tf.keras.regularizers.l2(weight_penalty_2))(layer3)

layer5 = tf.keras.layers.BatchNormalization()(layer4)
layer6 = tf.keras.layers.Dropout(drop_prob)(layer5)
layer7 = tf.keras.layers.Dense(16, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(weight_penalty_1),
                    activity_regularizer=tf.keras.regularizers.l2(weight_penalty_2))(layer6)

output = tf.keras.layers.Dense(1)(layer7)
model = tf.keras.Model(inputs=inputs, outputs=output)

In [ ]:
boundaries = [50, 200, 500, 1000]
values = [0.1, 0.05, 0.01, 0.005, 0.001]
learning_rate_fn = tf.keras.optimizers.schedules.PiecewiseConstantDecay(boundaries, values)
model_optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate_fn)
model.compile(
    loss='mape',
    optimizer=model_optimizer,
    metrics=['mae']
)

In [ ]:
from datetime import datetime
logdir = "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)
EarlyStoppingCallback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)

In [ ]:
class PriceModel:
    
    def __init__(self, dim, shape, drop_prob = 0.4, weight_penalty_1 = 1e-6, weight_penalty_2 = 1e-6):
        """
        Autoencoder for grey level images.

        Arguments:
            shape: shape of the input image [size_x, size_y, channels]
            filters: number of filters of the first conv layer
            latent_dim = size of the latent space (dimensions of the reduced space)
            lr: learning rate
            verbose = Boolean, if True, will print information about the models
        """
        
        self.input_dim = dim
        self.verbose = verbose

        # Build the Model
        inputs = tf.keras.layers.Input(shape=dim)
        x = inputs
        for layer in shape[:-1]:
          x = tf.keras.layers.Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(weight_penalty_1),
                            activity_regularizer=tf.keras.regularizers.l2(weight_penalty_2))(x)
          x = tf.keras.layers.BatchNormalization()(x)
          x = tf.keras.layers.Dropout(drop_prob)(x)
        
        x = tf.keras.layers.Dense(shape[-1], activation='relu', kernel_regularizer=tf.keras.regularizers.l2(weight_penalty_1),
                            activity_regularizer=tf.keras.regularizers.l2(weight_penalty_2))(x)
        output = tf.keras.layers.Dense(1)(x)

        self.model = tf.keras.Model(inputs=inputs, outputs=output)

        ## Compile it with an appropriate loss function
        logdir = "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
        tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)
        EarlyStoppingCallback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)
        boundaries = [50, 200, 500, 1000]
        values = [0.1, 0.05, 0.01, 0.005, 0.001]
        learning_rate_fn = tf.keras.optimizers.schedules.PiecewiseConstantDecay(boundaries, values)
        model_optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate_fn)
        self.model.compile(
            loss='mape',
            optimizer=model_optimizer,
            metrics=['mae']
        )
        self.model.summary()
        
    def fit(self, data, epochs, batch_size, validation_split=0.2):
        return self.autoencoder.fit(x=data, 
                             y=data, 
                             validation_split=validation_split,
                             shuffle=True, 
                             epochs=epochs, 
                             batch_size=batch_size)

    def predict(self, batch):
        """Autoencode batch of images"""
        return self.autoencoder.predict(batch)

    def save(self, path):
      self.autoencoder.save(path + '/my_autoencoder')
      self.encoder.save(path + '/my_encoder')
      self.decoder.save(path + '/my_decoder')

    def load_model(self, path):
      self.encoder = load_model(path + '/my_encoder')
      self.decoder = load_model(path + '/my_decoder')
      self.autoencoder = load_model(path + '/my_autoencoder')